In [45]:
import pandas as pd

In [46]:
train = pd.read_excel("/Users/FelipeGarcia/Downloads/machine.xls",sheetname=0,header=0)
train.shape

(891, 7)

In [47]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Parch,Embarked
0,1,0,3,male,22,0,S
1,2,1,1,female,38,0,C
2,3,1,3,female,26,0,S
3,4,1,1,female,35,0,S
4,5,0,3,male,35,0,S


In [48]:
test = pd.read_excel("/Users/FelipeGarcia/Downloads/machine2.xls",sheetname=0,header=0)
test.shape

(418, 6)

In [49]:
titanic = train
titanic['Sex_Female'] = (titanic['Sex'] == 'female').astype(np.int)
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix='Embarked')
embarked_dummies.drop(embarked_dummies.columns[0], axis=1, inplace=True)
titanic = pd.concat([titanic, embarked_dummies], axis=1)

In [50]:
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,Parch,Embarked,Sex_Female,Embarked_Q,Embarked_S
0,1,0,3,male,22,0,S,0,0,1
1,2,1,1,female,38,0,C,1,0,0
2,3,1,3,female,26,0,S,1,0,1
3,4,1,1,female,35,0,S,1,0,1
4,5,0,3,male,35,0,S,0,0,1


In [51]:
# define X and y
feature_cols = ['Pclass', 'Parch', 'Age', 'Sex_Female', 'Embarked_Q', 'Embarked_S']
X = titanic[feature_cols]
y = titanic.Survived

# train/test split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# train a logistic regression model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

# make predictions for testing set
y_pred_class = logreg.predict(X_test)

# calculate testing accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

0.77130044843


In [52]:
test['Sex_Female'] = (test['Sex'] == 'female').astype(np.int)
embarked_dummies = pd.get_dummies(test.Embarked, prefix='Embarked')
embarked_dummies.drop(embarked_dummies.columns[0], axis=1, inplace=True)
test = pd.concat([test, embarked_dummies], axis=1)

In [53]:
predictions = logreg.predict(test[feature_cols])

In [54]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': test['PassengerId'],
                            'Survived': predictions })
submission.to_csv("titanic_submission_CV.csv", index=False)